In [1]:
# %%capture
import IPython
import os
import numpy as np
from pandas import HDFStore
import spacy
from keras.utils import to_categorical
import cv2

from vqa_logger import logger
from common.os_utils import File

c:\local\Anaconda3-4.1.1-Windows-x86_64\envs\conda_env\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from common.constatns import train_data, validation_data, data_location, fn_meta, vqa_specs_location
from common.settings import input_length, embedding_dim, image_size, seq_length, get_nlp
from common.classes import VqaSpecs
from common.functions import get_highlited_function_code, get_image, get_text_features, pre_process_raw_data, get_size
from common.utils import VerboseTimer

In [3]:
meta_data = File.load_json(fn_meta)

### Preparing the data for training

#### Getting the nlp engine

In [4]:
nlp = get_nlp()

[13:52:13][DEBUG] using embedding vector: en_core_web_sm
[13:52:14][DEBUG] Got embedding


#### Where get_nlp is defined as:

In [5]:
code = get_highlited_function_code(get_nlp,remove_comments=True)
IPython.display.display(code)

In [6]:
from parsers.VQA18 import Vqa18Base
df_train = Vqa18Base.get_instance(train_data.processed_xls).data            
df_val = Vqa18Base.get_instance(validation_data.processed_xls).data

In [7]:
logger.debug('Building input dataframe')
cols = ['image_name', 'question', 'answer']

image_name_question = df_train[cols].copy()
image_name_question_val = df_val[cols].copy()



[13:52:15][DEBUG] Building input dataframe


##### This is just for performance and quick debug cycles! remove before actual trainining:

In [8]:
# image_name_question = image_name_question.head(5)
# image_name_question_val = image_name_question_val.head(5)

### Aditional functions we will use:

#### get_text_features:

In [9]:
code = get_highlited_function_code(get_text_features,remove_comments=True)
IPython.display.display(code)

#### get_image:

In [10]:
code = get_highlited_function_code(get_image,remove_comments=True)
IPython.display.display(code)

#### pre_process_raw_data:

In [11]:
code = get_highlited_function_code(pre_process_raw_data,remove_comments=True)
IPython.display.display(code)


#### This is for in case we want to classify by categorial labels (TBD):
(i.e. make it into a one large multiple choice test)

In [12]:
def get_categorial_labels(df, meta):
    ans_to_ix = meta['ans_to_ix']
    all_classes =  ans_to_ix.keys()
    data_classes = df['answer']
    class_count = len(all_classes)

    classes_indices = [ans_to_ix[ans] for ans in data_classes]
    categorial_labels = to_categorical(classes_indices, num_classes=class_count)
    
    for i in range(len(categorial_labels)):
        assert np.argmax(categorial_labels[i])== classes_indices[i], 'Expected to get argmax at index of label'
    


    return categorial_labels

with VerboseTimer("Getting categorial training labels"):
    categorial_labels_train = get_categorial_labels(df_train, meta_data)

with VerboseTimer("Getting categorial validation labels"):
    categorial_labels_val = get_categorial_labels(df_val, meta_data)
# categorial_labels_train.shape, categorial_labels_val.shape
del df_train
del df_val

Getting categorial training labels: 0:00:00.103983
Getting categorial validation labels: 0:00:00.010528


### Do the actual pre processing
Note:  
This might take a while...

In [13]:
logger.debug('----===== Preproceccing train data =====----')
image_locations = train_data.images_path
with VerboseTimer("Pre processing training data"):
    image_name_question = pre_process_raw_data(image_name_question, image_locations)

[13:52:15][DEBUG] ----===== Preproceccing train data =====----
[13:52:15][DEBUG] Getting questions embedding
[13:53:21][DEBUG] Getting answers embedding
[13:54:31][DEBUG] Getting image features
[13:55:03][DEBUG] Done
Pre processing training data: 0:02:47.535288


In [14]:
logger.debug('----===== Preproceccing validation data =====----')
image_locations = validation_data.images_path
with VerboseTimer("Pre processing validation data"):
    image_name_question_val = pre_process_raw_data(image_name_question_val, image_locations)

[13:55:03][DEBUG] ----===== Preproceccing validation data =====----
[13:55:03][DEBUG] Getting questions embedding
[13:55:10][DEBUG] Getting answers embedding
[13:55:17][DEBUG] Getting image features
[13:55:21][DEBUG] Done
Pre processing validation data: 0:00:17.873334


In [15]:
image_name_question.head(2)

,image_name,question,answer,path,question_embedding,answer_embedding,image
0,rjv03401.jpg,what does mri show?,lesion at tail of pancreas,C:\Users\Public\Documents\Data\2018\VQAMed2018...,"[[-1.8407480716705322, 2.5507988929748535, 0.7...","[[2.7199699878692627, 0.11310356855392456, -0....","[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
1,AIAN-14-313-g002.jpg,where does axial section mri abdomen show hypo...,in distal pancreas,C:\Users\Public\Documents\Data\2018\VQAMed2018...,"[[0.35850387811660767, 1.4076576232910156, -3....","[[1.1828632354736328, 0.4119483232498169, -3.4...","[[[9, 9, 9], [9, 9, 9], [10, 10, 10], [9, 9, 9..."


#### Saving the data, so later on we don't need to compute it again

In [16]:
def get_vqa_specs(meta_data):    
    dim = embedding_dim
    s_length = seq_length    
    return VqaSpecs(embedding_dim=dim, seq_length=s_length, data_location=data_location,meta_data=meta_data)

vqa_specs = get_vqa_specs(meta_data)

# Show waht we got...
s = str(vqa_specs)
s[:s.index('meta_data=')+10]

"VqaSpecs(embedding_dim=384, seq_length=26, data_location='C:\\\\Users\\\\avitu\\\\Documents\\\\GitHub\\\\VQA-MED\\\\VQA-MED\\\\Cognitive-LUIS-Windows-master\\\\Sample\\\\VQA.Python\\\\data\\\\model_input.h5', meta_data="

In [18]:

logger.debug("Saving the data")

item_to_save = image_name_question
# item_to_save = image_name_question.head(10)

# remove if exists
try:
    os.remove(data_location)
except OSError:
    pass

with VerboseTimer("Saving model training data"):
    with HDFStore(data_location) as store:
        store['train']  = image_name_question
        store['val']  = image_name_question_val
        

size = get_size(data_location)
logger.debug(f"training data's file size was: {size}")


item_to_save.to_hdf(vqa_specs.data_location, key='df')    
logger.debug(f"Saved to {vqa_specs.data_location}")

[13:56:45][DEBUG] Saving the data
Saving model training data: 0:00:17.265738
[13:57:02][DEBUG] training data's file size was: 1.92 GB
[13:57:18][DEBUG] Saved to C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\Cognitive-LUIS-Windows-master\Sample\VQA.Python\data\model_input.h5


In [20]:
File.dump_pickle(vqa_specs, vqa_specs_location)
logger.debug(f"VQA Specs saved to:\n{vqa_specs_location}")

[13:57:19][DEBUG] VQA Specs saved to:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\Cognitive-LUIS-Windows-master\Sample\VQA.Python\data\vqa_specs.pkl
